In [65]:
import pandas as pd
import numpy as np
import _pickle as pickle
import dtale
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model

In [32]:
stocks = pd.read_csv(r'C:\Users\Alex Willett\Desktop\research_tome\stockpup_train_data.csv')

In [33]:
stocks.head()

,Unnamed: 0,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,...,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio
0,0,0.060170,0.060170,0.0,3.011116,1.505420,2.212896,5.757136,3.477486,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.459442,0.000000,0.00000,-137.179487,-4.020246
1,1,0.164196,0.164196,0.0,6.911624,5.675302,9.182531,14.821592,5.601034,0.0,...,-40.465919,0.000000,0.0,0.0,0.0,-1.230090,0.000000,0.00000,17.241379,-7.966629
2,2,0.739888,0.739888,0.0,7.640220,5.819611,8.034188,9.880478,7.405141,0.0,...,-31.739130,0.000000,0.0,0.0,0.0,-0.223535,0.000000,0.00000,-144.117647,-3.696398
3,3,0.088384,0.088384,0.0,9.293769,-3.430592,16.044304,11.312545,5.242165,0.0,...,-2.335456,0.000000,0.0,0.0,0.0,-3.696591,0.000000,0.00000,-553.333333,-13.246196
4,4,0.066338,0.066338,0.0,-1.390096,-8.478301,-5.535860,0.912052,1.353546,0.0,...,-28.260870,-25.917859,0.0,0.0,0.0,2.775008,-8.163265,-6.47482,-100.000000,-9.304025


In [57]:
X = stocks.drop(['Price', 'Price high', 'Price low','Unnamed: 0'], 1)
y = stocks.Price

# Maximum numbers of features we need to select
num_feats = 15
feature_name = list(X.columns)

# how many features to choose from before selections
print(str(len(feature_name)), 'features to choose from')

37 features to choose from


In [39]:
d_X = dtale.show(X)

http://LAPTOP-DL3DBF8J:40001/dtale/main/2
It looks like this data may have already been loaded to D-Tale based on shape and column names. Here is URL of the data that seems to match it:

None

If you still want to load this data please use the following command:

dtale.show(df, ignore_duplicate=True)


Pearson Correlation

In [40]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

15 selected features


In [41]:
cor_feature

['Long-term debt to equity ratio',
 'Asset turnover',
 'Shares split adjusted',
 'Equity to assets ratio',
 'Dividend payout ratio',
 'Long-term debt',
 'Split factor',
 'ROA',
 'ROE',
 'Shares',
 'Liabilities',
 'Assets',
 'Shareholders equity',
 'Book value of equity per share',
 'P/B ratio']

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [94]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(100, activation='relu')(input_layer)
dense_layer_2 = Dense(50, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(25, activation='relu')(dense_layer_2)
output = Dense(1)(dense_layer_3)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

2020-04-08 15:37:26,107 - WARNING  - From C:\Users\Alex Willett\Anacondaweed\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
2020-04-08 15:37:26,369 - WARNING  - From C:\Users\Alex Willett\Anacondaweed\lib\site-packages\tensorflow\python\keras\utils\losses_utils.py:170: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


In [95]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(100, activation='relu')(input_layer)
dense_layer_2 = Dense(50, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(25, activation='relu')(dense_layer_2)
output = Dense(1)(dense_layer_3)

In [96]:
history = model.fit(X_train, y_train, batch_size=2, epochs=100, verbose=1, validation_split=0.2)

2020-04-08 15:37:52,164 - WARNING  - From C:\Users\Alex Willett\Anacondaweed\lib\site-packages\tensorflow\python\ops\math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Train on 35561 samples, validate on 8891 samples
Epoch 1/100
35561/35561 [==============================] - 25s 715us/sample - loss: 6912.9371 - mean_squared_error: 6912.8408 - val_loss: 3450.4548 - val_mean_squared_error: 3450.4565
Epoch 2/100
35561/35561 [==============================] - 24s 671us/sample - loss: 6922.9767 - mean_squared_error: 6922.9565 - val_loss: 3059.1693 - val_mean_squared_error: 3059.1694
Epoch 3/100
35561/35561 [==============================] - 24s 679us/sample - loss: 6372.9256 - mean_squared_error: 6372.8936 - val_loss: 2506.9780 - val_mean_squared_error: 2506.9761
Epoch 4/100
35561/35561 [==============================] - 25s 693us/sample - loss: 6113.8769 - mean_squared_error: 6113.7959 - val_loss: 2900.4452 - val_mean_squared_error: 2900.4414
Epoch 5/100
35561/35561 [==============================] - 28s 777us/sample - loss: 6608.7504 - mean_squared_error: 6608.7119 - val_loss: 2864.1900 - val_mean_squared_error: 2864.1951
Epoch 6/100
35561/35561 [======

2020-04-08 16:09:09,616 - INFO     - Executing shutdown due to inactivity...


35561/35561 [==============================] - 25s 701us/sample - loss: 3060.7269 - mean_squared_error: 3060.7087 - val_loss: 2177.8457 - val_mean_squared_error: 2177.8384
Epoch 79/100
15834/35561 [============>.................] - ETA: 11s - loss: 9918.8144 - mean_squared_error: 9918.8203 Executing shutdown...


2020-04-08 16:09:22,595 - INFO     - Executing shutdown...


35561/35561 [==============================] - 23s 655us/sample - loss: 4552.6154 - mean_squared_error: 4552.5532 - val_loss: 2489.6518 - val_mean_squared_error: 2489.6497
Epoch 80/100
35561/35561 [==============================] - 22s 628us/sample - loss: 5963.8657 - mean_squared_error: 5963.8462 - val_loss: 2939.4675 - val_mean_squared_error: 2939.4639
Epoch 81/100
35561/35561 [==============================] - 23s 656us/sample - loss: 4634.4773 - mean_squared_error: 4634.4414 - val_loss: 2173.8590 - val_mean_squared_error: 2173.8584
Epoch 82/100
35561/35561 [==============================] - 22s 624us/sample - loss: 5161.6402 - mean_squared_error: 5161.6343 - val_loss: 1889.7527 - val_mean_squared_error: 1889.7505
Epoch 83/100
35561/35561 [==============================] - 23s 642us/sample - loss: 3367.9308 - mean_squared_error: 3367.9199 - val_loss: 20027.1644 - val_mean_squared_error: 20027.1426
Epoch 84/100
35561/35561 [==============================] - 22s 630us/sample - loss: 5

In [97]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred_train = model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train)))

pred = model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, pred)))

55.37748995603209
3761.156311266845


In [98]:
score = model.evaluate(X_test, y_test, verbose=1)

print('Test Score:', score[0])
print('Test Accuracity', score[1])

11114/11114 [==============================] - 0s 27us/sample - loss: 14146296.4569 - mean_squared_error: 14146253.0000
Test Score: 14146296.456905723
Test Accuracity 14146253.0
